# Sajad Rahmanian 97101683

# Acknowledgement
This code is based on keras blog [post](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html) about sequence to sequence models.

# Importing Necessary Packages

In [53]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, Embedding, GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Getting the dataset ready

In [54]:
# Read the text file and split its lines:
with open("ferdousi.txt", 'r', encoding="utf-8") as f:
    text = f.read().splitlines()
# Remove the first two lines:
text = text[2:]
print(text[:5])
# Get the unique chars (add '\n' and '\t' to chars list)
joined_text = " ".join(text)
joined_text_d = joined_text + "\n\t"
chars = sorted(set(joined_text_d))
print(chars)
# Join mesraes (!) to create beyts (!)
m1 = text[0::2]
m2 = text[1::2]
beyts = [" ".join([x, y]) for x, y in zip(m1, m2)]
print(beyts[0])

['به نام خداوند جان و خرد', 'کزین برتر اندیشه برنگذرد', 'خداوند نام و خداوند جای', 'خداوند روزی ده رهنمای', 'خداوند کیوان و گردان سپهر']
['\t', '\n', ' ', '(', ')', '«', '»', '،', '؟', 'ء', 'آ', 'أ', 'ؤ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'ٔ', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']
به نام خداوند جان و خرد کزین برتر اندیشه برنگذرد


In [55]:
# number of unique chars (vocab_size)
n_chars = len(chars)
print(n_chars)

47


In [56]:
# maximum length of a beyt in dataset
max_beyt_len = max([len(x) for x in beyts])
print(max_beyt_len)

64


In [57]:
# define mappings from chars to their index and vice versa
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

In [58]:
# define input and output beyts
# for each input beyt, the output is its following beyt
# "\t" and "\n" represent the start and end of target outputs (this is needed for prediction)
input_texts = beyts[:-1]
output_texts = ["\t" + txt + "\n" for txt in beyts[1:]]
len(input_texts), len(output_texts)

(49607, 49607)

In [59]:
# map chars to their respective indices
# decoder_out is ahead of its input by one character
encoder_inputs_raw = [[char_to_idx[ch] for ch in sent] for sent in input_texts]
decoder_inputs_raw = [[char_to_idx[ch] for ch in sent] for sent in output_texts]
decoder_outputs_raw = [[char_to_idx[ch] for ch in sent[1:]] for sent in output_texts]

In [60]:
# pad the sequences to maximum length of beyts
# space char is used for padding
encoder_inputs_padded = pad_sequences(encoder_inputs_raw,
                                      maxlen=max_beyt_len,
                                      padding='post',
                                      value=char_to_idx[' '])
decoder_inputs_padded = pad_sequences(decoder_inputs_raw,
                                      maxlen=max_beyt_len,
                                      padding='post',
                                      value=char_to_idx[' '])
decoder_outputs_padded = pad_sequences(decoder_outputs_raw,
                                       maxlen=max_beyt_len,
                                       padding='post',
                                       value=char_to_idx[' '])

# Defining and training the end to end model

Read the comments for explanations

In [61]:
# PARAMS:
# Number of units in recurrent layers
RNN_UNITS = 1024
# Embedding dimension of chars
EMBEDDING_DIM = 256

In [62]:
# Get end to end model
def get_end_to_end_model(input_length=max_beyt_len,
                         vocab_size=n_chars, 
                         rnn_units=256,
                         embedding_dim=256,
                         rnn_type="LSTM"):

    # Input is a beyt with maximum length
    raw_enc_inputs = Input(shape=(input_length,))
    # Embed the input beyt to an embedding_dim size vector
    enc_embedder = Embedding(vocab_size, embedding_dim)
    enc_embedded_inputs = enc_embedder(raw_enc_inputs)

    if rnn_type == "LSTM":
        # Define recurrent layer. The output of encoder is not needed --> return_sequences=False
        enc_rnn_layer = LSTM(rnn_units, return_sequences=False, return_state=True)
        # Get the output state of the encoder
        _, enc_h, enc_c = enc_rnn_layer(enc_embedded_inputs)
        enc_states = [enc_h, enc_c]
    elif rnn_type == "GRU":
        enc_rnn_layer = GRU(rnn_units, return_sequences=False, return_state=True)
        _, enc_states = enc_rnn_layer(enc_embedded_inputs)
    else:
        raise NotImplemented

    # Input sequence of decoder
    raw_dec_inputs = Input(shape=(input_length,))
    # Embedding decoder input
    dec_embedder = Embedding(vocab_size, embedding_dim)
    dec_embedded_inputs = dec_embedder(raw_dec_inputs)

    if rnn_type == "LSTM":
        # Recurrent layer of decoder. We need its outputs --> return_sequences=True
        dec_rnn_layer = LSTM(rnn_units, return_sequences=True, return_state=True)
        dec_outputs, _, _ = dec_rnn_layer(dec_embedded_inputs, initial_state=enc_states)
    elif rnn_type == "GRU":
        dec_rnn_layer = GRU(rnn_units, return_sequences=True, return_state=True)
        dec_outputs, _ = dec_rnn_layer(dec_embedded_inputs, initial_state=enc_states)
    
    # From dec_outputs we need to predict the character 
    # --> the output size of the dense layer is equal to vocab_size (n_chars)
    dec_dense = Dense(vocab_size, activation="softmax")
    dec_output = dec_dense(dec_outputs)
    # For training the inputs of the model are: input beyt and target output beyt
    # The output of the model is the predicted output beyt
    model = Model([raw_enc_inputs, raw_dec_inputs], dec_output)

    return model    

# LSTM

In [63]:
# Get the model with LSTM
model_lstm = get_end_to_end_model(rnn_units=RNN_UNITS,
                                  embedding_dim=EMBEDDING_DIM,
                                  rnn_type="LSTM")

In [65]:
model_lstm.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model_lstm.fit(
    [encoder_inputs_padded, decoder_inputs_padded],
    decoder_outputs_padded,
    batch_size=64,
    epochs=10,
    validation_split=0.2,
)
model_lstm.save("end_to_end")

Epoch 1/10
621/621 [==============================] - 57s 88ms/step - loss: 1.4699 - accuracy: 0.5668 - val_loss: 1.2203 - val_accuracy: 0.6383
Epoch 2/10
621/621 [==============================] - 57s 91ms/step - loss: 1.0851 - accuracy: 0.6694 - val_loss: 1.1261 - val_accuracy: 0.6638
Epoch 3/10
621/621 [==============================] - 56s 90ms/step - loss: 1.0024 - accuracy: 0.6922 - val_loss: 1.0767 - val_accuracy: 0.6787
Epoch 4/10
621/621 [==============================] - 57s 92ms/step - loss: 0.9452 - accuracy: 0.7087 - val_loss: 1.0497 - val_accuracy: 0.6883
Epoch 5/10
621/621 [==============================] - 57s 91ms/step - loss: 0.9006 - accuracy: 0.7219 - val_loss: 1.0349 - val_accuracy: 0.6932
Epoch 6/10
621/621 [==============================] - 56s 91ms/step - loss: 0.8637 - accuracy: 0.7328 - val_loss: 1.0263 - val_accuracy: 0.6968
Epoch 7/10
621/621 [==============================] - 56s 90ms/step - loss: 0.8273 - accuracy: 0.7434 - val_loss: 1.0287 - val_accuracy:

In [66]:
test_out = model_lstm.predict(
    [encoder_inputs_padded[:1], decoder_inputs_padded[:1]]).argmax(2)
t = ""
for x in test_out[0]:
    t+= idx_to_char[x]
t

1/1 [==============================] - 1s 619ms/step


'اوایند پاپ ج نرایسد بزی نوایند بخز\n بگ وو م ی                   '

In [79]:
def get_enc_dec_models(e2e_model, rnn_units=256, rnn_type="LSTM"):
    # We need to separate the encoder and decoder for generating sequences

    # the first input of the model is the input to the encoder
    enc_inputs = e2e_model.input[0]
    if rnn_type == "LSTM":
      # Get encoder states from lstm layer
      _, enc_state_h_, enc_state_c_ = e2e_model.layers[4].output
      enc_states = [enc_state_h_, enc_state_c_]
      # Define the encoder with inputs defined as above,
      # and the outputs defined as its states
      enc_model = Model(enc_inputs, enc_states)
      # Decoder model input is the e2e model second input
      dec_inputs = e2e_model.input[1]
      # The decoder also gets its initial states from encoder or itself
      dec_input_states_h = Input((rnn_units,))
      dec_input_states_c = Input((rnn_units,))
      # Get the outputs of decoder lstm layer
      # Note that the input to the lstm layer should come from embedding layer
      dec_outputs, dec_output_states_h, dec_output_states_c = e2e_model.layers[5](
          e2e_model.layers[3](dec_inputs), # or e2e_model.layers[3](dec_inputs) # e2e_model.layers[3].output
          initial_state=[dec_input_states_h,dec_input_states_c])
      # Get dense layer's output
      dec_output = e2e_model.layers[6](dec_outputs)
      # Decoder inputs: inputs char, input states
      # Decoder outputs: pred_char_probs, output states
      dec_model = Model([dec_inputs,
                         dec_input_states_h,
                         dec_input_states_c],
                        [dec_output,
                         dec_output_states_h,
                         dec_output_states_c])

    elif rnn_type == "GRU":
        _, enc_states = e2e_model.layers[4].output
        enc_model = Model(enc_inputs, enc_states)
        dec_inputs = e2e_model.input[1]
        dec_input_states = Input((rnn_units,))
        dec_outputs, dec_output_states = e2e_model.layers[5](
            e2e_model.layers[3](dec_inputs),
            initial_state=dec_input_states)
        dec_output = e2e_model.layers[6](dec_outputs)
        dec_model = Model([dec_inputs,
                           dec_input_states],
                          [dec_output,
                           dec_output_states])
    else:
        raise NotImplemented

    return enc_model, dec_model

In [68]:
encoder_model, decoder_model = get_enc_dec_models(model_lstm, rnn_units=RNN_UNITS, rnn_type="LSTM")

# Generating function

When we want to generate a beyt from input beyt, we first feed the input beyt into encoder to get the initial state for decoder. Then, we use this initial state and start character ('\t') to predict next character. Then output state and the predicted character are fed to the decoder to predict next character, and so on.

In [84]:
def generate_beyt(input_beyt, encoder, decoder, rnn_type, generation_type="best"):
    # Set the first char to start char which is '\t'
    next_char = "\t"
    # Output beyt to be completed
    out_beyt = ""
    if rnn_type == "LSTM":
        # At first the decoder initial state should come from the encoder
        h_st, c_st = encoder(input_beyt, training=False)
        # We fill the output layer until we reach end of seq char or maximum length of beyts
        while next_char != "\n" and len(out_beyt) <= max_beyt_len:
        # The next initial states come from the decoder itself
            probs, h_st, c_st = decoder([np.array(char_to_idx[next_char], int).reshape((1,1)),
                                                 h_st,
                                                 c_st],
                                        training=False)
            probs = probs.numpy().flatten()
            if generation_type == "best":
                next_char = idx_to_char[probs.argmax()]
            elif generation_type == "sampling":
                next_char = idx_to_char[np.random.choice(range(len(probs)), p=probs)]
            # Add char to output
            out_beyt += next_char
    elif rnn_type == "GRU":
        # At first the decoder initial state should come from the encoder
        st = encoder(input_beyt, training=False)
        # We fill the output layer until we reach end of seq char or maximum length of beyts
        while next_char != "\n" and len(out_beyt) <= max_beyt_len:
        # The next initial states come from the decoder itself
            probs, st = decoder([np.array(char_to_idx[next_char], int).reshape((1,1)), st], training=False)
            probs = probs.numpy().flatten()
            if generation_type == "best":
                next_char = idx_to_char[probs.argmax()]
            elif generation_type == "sampling":
                next_char = idx_to_char[np.random.choice(range(len(probs)), p=probs)]
            # Add char to output
            out_beyt += next_char

    return out_beyt

## Two generation modes:
1. Use the most probable character each time (argmax)
2. Sample from the chars with probabilities equaling the softmax output

The second method is better for generating different outputs

In [74]:
test_indices = np.arange(len(encoder_inputs_padded))
np.random.shuffle(test_indices)
test_indices = test_indices[:10]
for idx in test_indices:
    in_seq = encoder_inputs_padded[idx:idx+1]
    out_seq = generate_beyt(in_seq, encoder_model, decoder_model, "LSTM", "best")
    in_seq_chars = ""
    for i in in_seq[0]:
        in_seq_chars += idx_to_char[i]
    print(f"Input Sequence: {in_seq_chars}")
    print(f"Output Sequence: {out_seq}")
    print("--------------------------------------")

Input Sequence: نهادند خوان پیش ایزدگشسب گرفتند پس واژ و برسم بدست              
Output Sequence: به پیش اندرون باره بردش نماز به نزدیک شاه آمد از بارگاه

--------------------------------------
Input Sequence: بران سان که شاهان نوازش کنند بران بندگان نیز نازش کنند          
Output Sequence: از گرد برنگذرد پیش او بر دل این داستان بر فزود

--------------------------------------
Input Sequence: نخواهم بدن زنده بی روی او جهانم نیرزد به یک موی او              
Output Sequence: به گفتار این نامدار اردشیر که با من به جنگ اندر آید سپید

--------------------------------------
Input Sequence: برآمد ز زاولستان رستخیز زمین خفته را بانگ برزد که خیز           
Output Sequence: از گرد برنگذرد پیش او بر دل این داستان بر فزود

--------------------------------------
Input Sequence: پیاده به پیش اندر افگند خوار به لشکرگه آوردش از کارزار          
Output Sequence: به پیش اندرون باره بردش نماز به نزدیک شاه آمد از بارگاه

--------------------------------------
Input Sequence: به ایران مرا کار زین بهترست همم

In [75]:
for idx in test_indices:
    in_seq = encoder_inputs_padded[idx:idx+1]
    out_seq = generate_beyt(in_seq, encoder_model, decoder_model, "LSTM", "sampling")
    in_seq_chars = ""
    for i in in_seq[0]:
        in_seq_chars += idx_to_char[i]
    print(f"Input Sequence: {in_seq_chars}")
    print(f"Output Sequence: {out_seq}")
    print("--------------------------------------")

Input Sequence: نهادند خوان پیش ایزدگشسب گرفتند پس واژ و برسم بدست              
Output Sequence: پس آنگه گرفت آنگهی انجمن گرفتند هر دو بوردگاه به زین

--------------------------------------
Input Sequence: بران سان که شاهان نوازش کنند بران بندگان نیز نازش کنند          
Output Sequence: راهی و تاج مهان

--------------------------------------
Input Sequence: نخواهم بدن زنده بی روی او جهانم نیرزد به یک موی او              
Output Sequence: مجو ایمن از کار من چون بهشت چنان دان که بر خیزد آن فر و برگ

--------------------------------------
Input Sequence: برآمد ز زاولستان رستخیز زمین خفته را بانگ برزد که خیز           
Output Sequence: از شرم نیزه وراند پذیره شدنده ز آتش بره برنداشت

--------------------------------------
Input Sequence: پیاده به پیش اندر افگند خوار به لشکرگه آوردش از کارزار          
Output Sequence: همان نیز شادی بر اسبان شدند بد و بوم و آخوبی و نیست مرد گرفت آگهی
--------------------------------------
Input Sequence: به ایران مرا کار زین بهترست همم کردگار جهان یاورست  

# GRU

In [76]:
# Get the model with GRU
model_gru = get_end_to_end_model(rnn_units=RNN_UNITS,
                                  embedding_dim=EMBEDDING_DIM,
                                  rnn_type="GRU")

In [77]:
model_gru.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history_gru = model_gru.fit(
    [encoder_inputs_padded, decoder_inputs_padded],
    decoder_outputs_padded,
    batch_size=64,
    epochs=10,
    validation_split=0.2,
)
model_gru.save("end_to_end_gru")

Epoch 1/10
621/621 [==============================] - 50s 76ms/step - loss: 1.7763 - accuracy: 0.4977 - val_loss: 1.4043 - val_accuracy: 0.5894
Epoch 2/10
621/621 [==============================] - 46s 74ms/step - loss: 1.2053 - accuracy: 0.6387 - val_loss: 1.1810 - val_accuracy: 0.6507
Epoch 3/10
621/621 [==============================] - 47s 75ms/step - loss: 1.0599 - accuracy: 0.6774 - val_loss: 1.1196 - val_accuracy: 0.6683
Epoch 4/10
621/621 [==============================] - 46s 75ms/step - loss: 0.9844 - accuracy: 0.6985 - val_loss: 1.0794 - val_accuracy: 0.6799
Epoch 5/10
621/621 [==============================] - 46s 74ms/step - loss: 0.9319 - accuracy: 0.7137 - val_loss: 1.0589 - val_accuracy: 0.6878
Epoch 6/10
621/621 [==============================] - 46s 74ms/step - loss: 0.8908 - accuracy: 0.7256 - val_loss: 1.0517 - val_accuracy: 0.6922
Epoch 7/10
621/621 [==============================] - 46s 74ms/step - loss: 0.8444 - accuracy: 0.7392 - val_loss: 1.0495 - val_accuracy:

In [80]:
encoder_model_gru, decoder_model_gru = get_enc_dec_models(model_gru, rnn_units=RNN_UNITS, rnn_type="GRU")

In [85]:
# test_indices = np.arange(len(encoder_inputs_padded))
# np.random.shuffle(test_indices)
# test_indices = test_indices[:10]
for idx in test_indices:
    in_seq = encoder_inputs_padded[idx:idx+1]
    out_seq = generate_beyt(in_seq, encoder_model_gru, decoder_model_gru, "GRU", "best")
    in_seq_chars = ""
    for i in in_seq[0]:
        in_seq_chars += idx_to_char[i]
    print(f"Input Sequence: {in_seq_chars}")
    print(f"Output Sequence: {out_seq}")
    print("--------------------------------------")

Input Sequence: نهادند خوان پیش ایزدگشسب گرفتند پس واژ و برسم بدست              
Output Sequence: به پیش سپاه اندرون با کمر همی خواست کاید به تنگی فراز

--------------------------------------
Input Sequence: بران سان که شاهان نوازش کنند بران بندگان نیز نازش کنند          
Output Sequence: به دژ بر یکی بانگ برزد به خون به دیدار او بر تن آسان شون

--------------------------------------
Input Sequence: نخواهم بدن زنده بی روی او جهانم نیرزد به یک موی او              
Output Sequence: به پیش سپاه اندرون با کمر همی خواست کاید به تنگی فراز

--------------------------------------
Input Sequence: برآمد ز زاولستان رستخیز زمین خفته را بانگ برزد که خیز           
Output Sequence: به دژ بر یکی بانگ برزد به خون به دیدار او بر تن آسان شون

--------------------------------------
Input Sequence: پیاده به پیش اندر افگند خوار به لشکرگه آوردش از کارزار          
Output Sequence: به دژ بر یکی بانگ برزد به خون به دیدار او بر تن آسان شون

--------------------------------------
Input Sequence: به ایران مرا کا

In [86]:
for idx in test_indices:
    in_seq = encoder_inputs_padded[idx:idx+1]
    out_seq = generate_beyt(in_seq, encoder_model_gru, decoder_model_gru, "GRU", "sampling")
    in_seq_chars = ""
    for i in in_seq[0]:
        in_seq_chars += idx_to_char[i]
    print(f"Input Sequence: {in_seq_chars}")
    print(f"Output Sequence: {out_seq}")
    print("--------------------------------------")

Input Sequence: نهادند خوان پیش ایزدگشسب گرفتند پس واژ و برسم بدست              
Output Sequence: پدر چنگ چندی برفت و به رود همی لشکر و دشت بارآلگوش

--------------------------------------
Input Sequence: بران سان که شاهان نوازش کنند بران بندگان نیز نازش کنند          
Output Sequence: که تو شاه را یک به یکی درور چه چاچیز مانیم خوبی و گوش

--------------------------------------
Input Sequence: نخواهم بدن زنده بی روی او جهانم نیرزد به یک موی او              
Output Sequence: چنین داد پاسخ که این داستان مگر پاک ماند به گرمان مکاش

--------------------------------------
Input Sequence: برآمد ز زاولستان رستخیز زمین خفته را بانگ برزد که خیز           
Output Sequence: یکی شارستان این بر نیمروز برافراخته سر پر از خون زوار

--------------------------------------
Input Sequence: پیاده به پیش اندر افگند خوار به لشکرگه آوردش از کارزار          
Output Sequence: یکی شارستان نام بر پهلوی که رخشنده شمع و شب آباد چوی

--------------------------------------
Input Sequence: به ایران مرا کار زین بهترست